# Simple LLM Chain

### Put Your OPENROUTER_API_KEY here

In [ ]:
import os
os.environ["OPENROUTER_API_KEY"] = "paste_your_api_key_here"

In [ ]:
!pip3 install langchain langchain_openai

In [ ]:
import os
from langchain_openai import ChatOpenAI

try:
    model_name
except NameError:
    model_name="openai/gpt-4o-mini"

print("Model Name:", model_name)
print("Provider:", "OpenRouter AI")

llm=ChatOpenAI(model_name=model_name,
               openai_api_key=os.environ.get("OPENROUTER_API_KEY"),
               openai_api_base="https://openrouter.ai/api/v1")

In [ ]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate

from IPython.display import display, clear_output, Markdown
from ipywidgets import widgets, Layout

conversation_output = widgets.Output()
messages = []

def run_chatbot(system_prompt, initial_message):
    global messages 
    messages = [ {'role':'system', 'content': system_prompt} ]
    conversation_output.clear_output()

    messages.append({'role': 'assistant', 'content': initial_message})

    text_input = widgets.Text(
        placeholder='Type your message here...',
        layout=widgets.Layout(width='50%')
    )
    submit_button = widgets.Button(description="Send")

    input_box = widgets.HBox([text_input, submit_button])
    display(conversation_output, input_box)

    def on_submit_click(b):
        message = text_input.value
        text_input.value = ''  # Clear the input field

        with conversation_output:
            display(Markdown(f"**User**: {message}"))
            messages.append({'role': 'user', 'content': message})
            response = get_completion_messages(messages)
            display(Markdown(f"**AI**: {response}"))
            messages.append({'role': 'assistant', 'content': response})

    submit_button.on_click(on_submit_click)

    # Display initial AI message
    with conversation_output:
        display(Markdown(f"**AI**: {initial_message}"))

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    text = text.lstrip()
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_messages():
    for message in messages:
        role = message['role']
        content = message['content']
        
        if role == 'system':
            print("System:")
            print("-" * 40)
            print(content)
        elif role == 'user':
            print("User: ", end="")
            print(wrap_text(content))
        elif role == 'assistant':
            print("Assistant: ", end="")
            print(wrap_text(content))
        print()  # Add an extra newline for spacing

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(wrap_text(prompt))
    print("")
    print("Response: ")
    print(response)

def print_messages_and_response(messages, response):
    prompt = ChatPromptTemplate(messages=messages)
    print_prompt_and_response(prompt.format(), response)

def get_completion(prompt, temperature=0.0):
    response = llm.invoke(prompt, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

def get_completion_messages(messages, temperature=0.0):
    response=llm.invoke(messages, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

### Description of Simple LLM Chain

Here we build a simple LLM Chain that makes two calls to the LLM. The
second step depends on the answer from the first step. This shows the
idea of a chain.

### Step 1: What is the city {person} is from?

In [ ]:
city_template="""
Where is {person} from? Give just the city name
"""

city_prompt = city_template.format(person="Sean Connery")
city_response = get_completion(city_prompt)
print(city_response)

### Step 2: What country is {city} in?

In [ ]:
country_template="""
What country is the city {city} in? Respond with just the name of the country 
"""

country_prompt = country_template.format(city=city_response)
country_response = get_completion(country_prompt)
print(country_response)

### Step 3: Final Report

Sometimes we will want to print out a summary of the responses we get in
a chain. This may or may not include the prompts we give as well. We can
just use a template for this as well, even though this is not a prompt,
it is a convenient way to set up a printed version of the information.

In [ ]:
report_template="""
## Final Report

### Person
{person}

### City
{city}

### Country
{country}
"""

# We pass the format function the values we used or got from the LLM
report = report_template.format(person = "Sean Connery",
                                city = city_response,
                                country= country_response)
print(report)